<a href="https://colab.research.google.com/github/tazar09/cars_dimensions/blob/main/comparison_02-feb2024.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import re

import warnings
warnings.filterwarnings('ignore')

# Scrapping the website

In [2]:
url = 'https://www.automobiledimension.com/compact-suv.php'
h3 = requests.get(url, verify = False)
html_content = h3.text

# Creating BeautifulSoup objects

In [3]:
soup = BeautifulSoup(html_content, 'html.parser')
main_table = soup.find('main')

In [4]:
temp_list = []
for i in main_table.find_all('div', class_ = 'unit'):
  link = str(i.a['href'])
  temp_list.append('https://www.automobiledimension.com' + link)
temp_list[0:3]

['https://www.automobiledimension.com/model/ford/puma',
 'https://www.automobiledimension.com/model/audi/q2',
 'https://www.automobiledimension.com/model/nissan/juke']

# Extract model description

In [6]:
def extract_main_text(text):
  search = text.find('div', class_ = 'interior-text')
  return search.text

In [7]:
def extract_boot_space(text):
  search = text.find_all('figcaption')[1]
  boot_space = search.text
  return boot_space

In [8]:
main_text_list = []
boot_space_list = []
for i in temp_list:
  content = requests.get(i, verify = False)
  content = BeautifulSoup(content.text, 'html.parser')
  main_text_list.append(extract_main_text(content))
  boot_list.append(extract_boot_space(content))

In [15]:
df = pd.DataFrame({'description': main_text_list, 'boot_space': boot_list})
df.head(5)

,description,boot_space
0,"The Ford Puma has a length of 4207 mm, a heigh...",Boot space: 401* - 456 liters.
1,"The Audi Q2 has a length of 4208 mm, a height ...",Boot space: 405 liters.
2,"The Nissan Juke has a length of 4210 mm, a hei...",Boot space: 354* - 422 liters.
3,"The Opel Crossland has a length of 4212 mm, a ...",Boot space: 410 liters.
4,"The SsangYong Tivoli has a length of 4225 mm, ...",Boot space: 427 liters.


In [20]:
def average(text):
  min, max = text.split('-')
  average = (int(max) + int(min))/2
  return average

df['boot_space'] = df['boot_space'].str.replace('Boot space: ', '')
df['boot_space'] = df['boot_space'].str.replace(' liters.','')
df['boot_space'] = df['boot_space'].apply(lambda x: x.replace('* - ', '-') if len(x)>3 else x)
df['boot_space'] = df['boot_space'].apply(lambda x: average(x) if len(x) > 3 else int(x))

df.head()

,description,boot_space
0,"The Ford Puma has a length of 4207 mm, a heigh...",428.5
1,"The Audi Q2 has a length of 4208 mm, a height ...",405.0
2,"The Nissan Juke has a length of 4210 mm, a hei...",388.0
3,"The Opel Crossland has a length of 4212 mm, a ...",410.0
4,"The SsangYong Tivoli has a length of 4225 mm, ...",427.0


# Extract dimensions

## Function for extracting

In [ ]:
columns = ["length", "height", "width", "measurement", "boot space", "ground clearance"]
for i in columns:
  df[i]=df['description'].apply(lambda x: extract_car_info)

In [ ]:
models[0].apply(lambda x: extract_car_data)